In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import scrapbook as sb 
import pandas as pd
import numpy as np 
import seaborn as sns 
import numpy as np
from statistics import mean
import matplotlib.pyplot as plt

# Baseline 

In [3]:
books = sb.read_notebooks("./BaseLine_Model_Output")
baseLine_data = []
for nb in books.notebooks: 
    nbList=[nb.scraps['Stats Model MAE'].data,nb.scraps['Catboost MAE'].data]
    baseLine_data.append(nbList)
df = pd.DataFrame(baseLine_data, columns = ["Stats Model","Catboost"])
#baseLine_data = np.reshape(baseLine_data,(2,10))
display(df)
print(df.mean())

,Stats Model,Catboost
0,0.206277,0.019123
1,0.263053,0.042150
2,0.294179,0.116115
3,0.290315,0.175373
4,0.252740,0.040753
5,0.216558,0.041667
6,0.230875,0.028887
7,0.339092,0.056652
8,0.281234,0.079000
9,0.297648,0.020032


Stats Model    0.267197
Catboost       0.061975
dtype: float64


# GAN Model

In [4]:
book = sb.read_notebooks("./GAN_Output")
gan_data = []
gan_mse = []
for nb in book.notebooks:
    metrics = nb.scraps['GAN_1 Metrics'].data
    for i in range(1000):
        gan_mse.append(metrics[0][i])
    nbList = [nb.scraps['GAN Model MSE'].data,
              nb.scraps['GAN Model MAE'].data,
              nb.scraps['GAN Model Euclidean distance'].data,
              nb.scraps['GAN Model Manhattan Distance'].data]        
    gan_data.append(nbList)
    
df = pd.DataFrame(gan_data, columns = ['MSE','MAE','Euclidean Distance','Manhattan Distance'])
display(df.style)
print("MEAN:")
print(df.mean(axis = 0))
gan_data = np.array(gan_data)

,MSE,MAE,Euclidean Distance,Manhattan Distance
0,0.003925,0.046379,0.278253,0.927583
1,0.015190,0.090338,0.550521,1.806768
2,0.003680,0.050162,0.270881,1.003233
3,0.002585,0.039138,0.226374,0.782752
4,0.010736,0.082382,0.462688,1.647635
5,0.004975,0.057724,0.315023,1.154474
6,0.006896,0.066599,0.370883,1.331986
7,0.004594,0.050541,0.302654,1.010813
8,0.007402,0.065142,0.384195,1.302832
9,0.007516,0.063800,0.385625,1.275995


MEAN:
MSE                   0.006750
MAE                   0.061220
Euclidean Distance    0.354710
Manhattan Distance    1.224407
dtype: float64


# ABC_GAN Analysis

## ABC Pre-generator - Catboost 


In [5]:
book = sb.read_notebooks("./ABC_GAN_Catboost")
paramVal = [1,0.1,0.01]
abc_mae = [[] for i in range(3)]
abc_mae_skip = [[] for i in range(3)]
abc_mae_mean = [[] for i in range(3)]
abc_mae_skip_mean = [[] for i in range(3)]
abc_weights = [[] for i in range(3)]
prior_model = [[] for i in range(3)]
abc_pre_generator = [[] for i in range(3)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    
    #Divide data according to parameters 
    for i in range(3):
        if paramVar == paramVal[i]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [6]:
for i in range(3):
    data = []
    for j in range(len(abc_weights[i])):
        data.append([paramVal[i],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    
    df = pd.DataFrame(data, columns = ['Variance','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.mean(axis=0))
    print("-------------------------------------------------------------")

,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,0.10788,0.76100,0.11782,0.05104,0.04500
1,1,0.20434,0.90008,0.77668,0.08154,0.06273
2,1,0.10045,0.80006,0.60629,0.05128,0.06920
3,1,0.12839,0.70060,0.17109,0.07111,0.07460
4,1,0.13194,1.17897,0.12581,0.04023,0.06413
5,1,0.13342,0.87695,0.11167,0.07748,0.06036
6,1,0.18659,0.78768,0.70052,0.05928,0.06252
7,1,0.22818,0.76508,0.17886,0.09258,0.06739
8,1,0.16487,0.94808,0.46595,0.07319,0.08841
9,1,0.12184,0.69085,0.34320,0.07653,0.05635


Variance                         1.000000
Prior Model MAE                  0.150789
ABC pre-generator MAE            0.840935
Skip Node weight                 0.359790
ABC GAN MAE                      0.067426
ABC_GAN MAE (skip connection)    0.065070
dtype: float64
-------------------------------------------------------------


,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.1,0.08610,0.10649,0.24291,0.05106,0.05227
1,0.1,0.14578,0.17192,0.19878,0.07733,0.06649
2,0.1,0.27600,0.32599,0.12608,0.15341,0.16496
3,0.1,0.20339,0.23446,0.16555,0.12026,0.12518
4,0.1,0.11525,0.17474,0.15023,0.08733,0.06879
5,0.1,0.11410,0.12623,0.76903,0.06090,0.06627
6,0.1,0.19848,0.21207,0.63820,0.12676,0.10630
7,0.1,0.17982,0.17983,0.08306,0.06794,0.07698
8,0.1,0.15664,0.18423,0.11701,0.05735,0.07405
9,0.1,0.09684,0.13393,0.50819,0.06954,0.04589


Variance                         0.100000
Prior Model MAE                  0.157241
ABC pre-generator MAE            0.184988
Skip Node weight                 0.299905
ABC GAN MAE                      0.087187
ABC_GAN MAE (skip connection)    0.084718
dtype: float64
-------------------------------------------------------------


,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.13239,0.13099,0.00000,0.12973,0.13319
1,0.01,0.14519,0.14465,0.01253,0.06455,0.14832
2,0.01,0.08648,0.08992,0.73053,0.05710,0.07577
3,0.01,0.18924,0.19195,0.00000,0.13691,0.18974
4,0.01,0.17649,0.17745,0.05321,0.13017,0.16263
5,0.01,0.12250,0.12316,0.03582,0.09453,0.12250
6,0.01,0.20298,0.20220,0.24620,0.13170,0.23248
7,0.01,0.09669,0.09531,0.00000,0.07266,0.09735
8,0.01,0.12978,0.13138,0.00000,0.08713,0.13023
9,0.01,0.05960,0.05845,0.03325,0.09965,0.05914


Variance                         0.010000
Prior Model MAE                  0.134132
ABC pre-generator MAE            0.134546
Skip Node weight                 0.111155
ABC GAN MAE                      0.100414
ABC_GAN MAE (skip connection)    0.135135
dtype: float64
-------------------------------------------------------------


## ABC Pre-generator - Stats 


In [7]:
book = sb.read_notebooks("./ABC_GAN_Stats")
paramVal = [1,0.1,0.01]
abc_mae = [[] for i in range(3)]
abc_mae_skip = [[] for i in range(3)]
abc_mae_mean = [[] for i in range(3)]
abc_mae_skip_mean = [[] for i in range(3)]
abc_weights = [[] for i in range(3)]
prior_model = [[] for i in range(3)]
abc_pre_generator = [[] for i in range(3)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    
    #Divide data according to parameters 
    for i in range(3):
        if paramVar == paramVal[i]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [8]:
for i in range(3):
    data = []
    for j in range(len(abc_weights[i])):
        data.append([paramVal[i],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    
    df = pd.DataFrame(data, columns = ['Variance','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.mean(axis=0))
    print("-------------------------------------------------------------")

,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,0.28921,0.63763,0.85161,0.08383,0.06319
1,1,0.25018,0.65117,0.85935,0.09515,0.07222
2,1,0.32242,0.85269,0.95050,0.08791,0.03898
3,1,0.26235,0.81121,0.89665,0.16655,0.04544
4,1,0.21099,0.68084,0.97935,0.10690,0.07061
5,1,0.21117,0.73699,0.91408,0.04837,0.07332
6,1,0.29112,0.99918,0.97004,0.09318,0.05352
7,1,0.26079,0.90658,0.91450,0.07810,0.06554
8,1,0.24676,0.77659,0.84250,0.09461,0.04639
9,1,0.26143,1.14343,0.85480,0.11415,0.04475


Variance                         1.000000
Prior Model MAE                  0.260642
ABC pre-generator MAE            0.819632
Skip Node weight                 0.903338
ABC GAN MAE                      0.096873
ABC_GAN MAE (skip connection)    0.057394
dtype: float64
-------------------------------------------------------------


,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.1,0.34432,0.33676,0.17433,0.09368,0.04923
1,0.1,0.31233,0.29145,0.37617,0.10208,0.03134
2,0.1,0.28497,0.33115,0.21448,0.09397,0.03947
3,0.1,0.43792,0.45244,0.19750,0.14619,0.05760
4,0.1,0.27773,0.29404,0.18846,0.10421,0.04965
5,0.1,0.37456,0.39479,0.33893,0.09203,0.06755
6,0.1,0.31085,0.29013,0.26912,0.05988,0.04297
7,0.1,0.29824,0.32685,0.22959,0.07756,0.05399
8,0.1,0.19577,0.22169,0.16420,0.10572,0.03996
9,0.1,0.33483,0.38819,0.23619,0.10192,0.06850


Variance                         0.100000
Prior Model MAE                  0.317153
ABC pre-generator MAE            0.332750
Skip Node weight                 0.238897
ABC GAN MAE                      0.097724
ABC_GAN MAE (skip connection)    0.050027
dtype: float64
-------------------------------------------------------------


,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.40060,0.39975,0.13826,0.10040,0.28486
1,0.01,0.33354,0.33284,0.11976,0.09927,0.07045
2,0.01,0.29560,0.29768,0.11707,0.09921,0.09135
3,0.01,0.22913,0.22901,0.13696,0.09848,0.04657
4,0.01,0.17458,0.17201,0.14208,0.09638,0.05697
5,0.01,0.28481,0.28361,0.16206,0.13889,0.06364
6,0.01,0.40926,0.40535,0.11268,0.07469,0.07512
7,0.01,0.31863,0.31712,0.19647,0.16729,0.06359
8,0.01,0.30141,0.29938,0.16383,0.12697,0.05477
9,0.01,0.22362,0.21858,0.20322,0.13462,0.04799


Variance                         0.010000
Prior Model MAE                  0.297117
ABC pre-generator MAE            0.295533
Skip Node weight                 0.149238
ABC GAN MAE                      0.113621
ABC_GAN MAE (skip connection)    0.085531
dtype: float64
-------------------------------------------------------------
